In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import pathlib



In [35]:
exp_dir = "experiments/191023_174532_fold_model_type_film_test_run_all_folds/*"
exp_dir = "experiments/191026_141145_fold_seed_model_type_test_two_seeds/*"
exp_dir = "experiments/191028_204715_gamma_steplr_fold_num_epoch_lr_bs_model_type_seed_test_restartable/*"

In [40]:
class HyperParametersSummary:
    def __init__(self, filter_path: str):
        self.predictions = []
        self.parameters = []
        self.table = []
        for f in tqdm(glob(filter_path)):
            parameters = f.split('/')[-1]
            self.parameters.append(parameters)
            column = {}
            for parameter in parameters.split(','):                
                parameter_key, parameter_value = parameter.split('=')
                column[parameter_key] = parameter_value
            # Insert some code here to parse the metrics and their values
            test_file = os.path.join(f, 'testing', "fo=%s" %column["fold"], "final_test_miou.txt")
            if os.path.isfile(test_file):
                with open(test_file) as f:
                    content = f.readlines()
                    for line in content:
                        key, value = line.split(':')
                        column[key] = float(value)
            else:
                pass
            self.table.append(column)
        self.table = pd.DataFrame(self.table)
        self.group_names = ', '.join(self.table.columns)
        print(f'Parameters: {self.group_names}')


In [45]:
hp_summary = HyperParametersSummary(exp_dir)

# hp_summary.table.groupby(['model_type', 'film', 'fold']).mean().unstack()
# hp_summary.table.groupby(['model_type', 'seed', 'fold']).mean().unstack()
hp_summary.table.groupby(['model_type', 'num_epoch', 'gamma_steplr', 'lr', 'bs']).mean().unstack()

100%|██████████| 108/108 [00:00<00:00, 1353.09it/s]

Parameters: fold, gamma_steplr, num_epoch, lr, bs, model_type, seed, IOU


IOU            
bs                                                  16   4       8
model_type     num_epoch gamma_steplr lr                          
iter_nwe_coatt 120       0.1          0.000125     NaN NaN     NaN
                                      0.00025      NaN NaN     NaN
                                      0.0005       NaN NaN     NaN
                         0.5          0.000125     NaN NaN     NaN
                                      0.00025      NaN NaN     NaN
                                      0.0005       NaN NaN     NaN
               180       0.1          0.000125     NaN NaN     NaN
                                      0.00025      NaN NaN     NaN
                                      0.0005       NaN NaN     NaN
                         0.5          0.000125     NaN NaN     NaN
                                      0.00025      NaN NaN     NaN
                                      0.0005       NaN NaN     NaN
               60        0.1          0.000125     NaN NaN     NaN
                                      0.00025      NaN NaN     NaN
                                      0.0005       NaN NaN     NaN
                         0.5          0.000125     NaN NaN     NaN
                                      0.00025      NaN NaN     NaN
                                      0.0005       NaN NaN     NaN
nwe            120       0.1          0.000125     NaN NaN     NaN
                                      0.00025      NaN NaN     NaN
                                      0.0005       NaN NaN     NaN
                         0.5          0.000125     NaN NaN     NaN
                                      0.00025      NaN NaN     NaN
                                      0.0005       NaN NaN     NaN
               180       0.1          0.000125     NaN NaN     NaN
                                      0.00025      NaN NaN     NaN
                                      0.0005       NaN NaN     NaN
                         0.5          0.000125     NaN NaN     NaN
                                      0.00025      NaN NaN     NaN
                                      0.0005       NaN NaN     NaN
               60        0.1          0.000125  0.3171 NaN  0.3693
                                      0.00025   0.3491 NaN     NaN
                                      0.0005    0.4543 NaN  0.4514
                         0.5          0.000125     NaN NaN     NaN
                                      0.00025      NaN NaN     NaN
                                      0.0005       NaN NaN     NaN

In [34]:
results_aggregated = hp_summary.table.groupby(['model_type', 'seed', 'fold']).mean().unstack()
results_aggregated['mIOU'] = results_aggregated.mean(axis=1)
results_aggregated

IOU                              mIOU
fold                         0       1       2       3          
model_type     seed                                             
coatt          120937      NaN  0.5885  0.4614     NaN  0.524950
               5982340  0.4540  0.5986  0.4719  0.4613  0.496450
iter_nwe_coatt 120937      NaN     NaN  0.4870     NaN  0.487000
               5982340     NaN     NaN  0.5054     NaN  0.505400
nwe            120937   0.4819  0.6498  0.4882  0.4413  0.515300
               5982340  0.5285  0.6521  0.4742  0.4573  0.528025
nwe_coatt      120937      NaN     NaN  0.4954     NaN  0.495400
               5982340  0.4804  0.6542  0.4969     NaN  0.543833

In [6]:
results_aggregated = hp_summary.table.groupby(['model_type', 'film', 'fold']).mean().unstack()
results_aggregated['mIOU'] = results_aggregated.mean(axis=1)
results_aggregated

IOU                              mIOU
fold                      0       1       2       3          
model_type     film                                          
coatt          0     0.4657  0.5941  0.4761  0.4384  0.493575
               1     0.4425  0.5838  0.4780  0.4259  0.482550
iter_nwe_coatt 0     0.5004  0.6537  0.4866  0.4765  0.529300
               1     0.4654  0.6474  0.4991  0.4760  0.521975
nwe            0     0.4766  0.6469  0.4904  0.4723  0.521550
               1     0.5081  0.6472  0.4604  0.4697  0.521350
nwe_coatt      0     0.4679  0.6408  0.4785  0.4662  0.513350
               1     0.4484  0.5929  0.4542  0.3734  0.467225